# comercio_ext_estatisticas.tb_importacoes
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/IMP_*_delta`

## 📌 Descrição do arquivo

Base de **importações brasileiras**, agregada mensalmente, por produto, país de origem, estado de destino e características logísticas e aduaneiras.  
Não contém identificação de empresas ou CNPJ.

Fonte: MDIC / Comex Stat  
Tipo: Dado público, agregado, não sensível

| Coluna       | Descrição técnica      | Interpretação humana                     |
| ------------ | ---------------------- | ---------------------------------------- |
| `CO_ANO`     | Ano da operação        | Ano em que a importação ocorreu          |
| `CO_MES`     | Mês da operação        | Mês de referência da importação          |
| `CO_NCM`     | Código NCM             | Produto importado                        |
| `CO_UNID`    | Unidade estatística    | Unidade oficial usada para o produto     |
| `CO_PAIS`    | Código do país         | País de origem da mercadoria             |
| `SG_UF_NCM`  | Sigla da UF            | Estado brasileiro de destino             |
| `CO_VIA`     | Código da via          | Meio de transporte utilizado             |
| `CO_URF`     | Código da URF          | Unidade da Receita Federal responsável   |
| `QT_ESTAT`   | Quantidade estatística | Quantidade na unidade estatística do NCM |
| `KG_LIQUIDO` | Peso líquido (kg)      | Peso total importado                     |
| `VL_FOB`     | Valor FOB (US$)        | Valor da mercadoria importada            |
| `VL_FRETE`   | Valor do frete (US$)   | Custo de transporte internacional        |
| `VL_SEGURO`  | Valor do seguro (US$)  | Custo do seguro internacional            |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzeBaseDir =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/"   
val silverPath =        "/mnt/silver/landingbeca2026jan/comercio_ext_estatisticas/tb_importacoes/"   
val silverTable =       "tb_importacoes"
val prefix =            "IMP_"
val suffix =            "_delta"

val fs = FileSystem . get (
  spark.sparkContext.hadoopConfiguration
)

val sourceDirs: Seq [ String ] = fs 
    . listStatus  ( new Path ( bronzeBaseDir )
) . filter      ( _ . isDirectory
) . map         ( _ . getPath . getName
) . filter      ( name => name . matches ( s"${prefix}\\d{4}${suffix}" )
) . toSeq

require (
  sourceDirs . nonEmpty , 
  s"Nenhuma pasta $prefix encontrada em $bronzeBaseDir"
)

val sourcePaths: Seq [ String ] = sourceDirs . map (
  d => s"$bronzeBaseDir/$d"
)

In [0]:
val silverSchema = StructType(
	Seq (
    StructField ( "CO_ANO" ,      IntegerType,              nullable = false ) ,
    StructField ( "CO_MES" ,      IntegerType,              nullable = false ) ,
		StructField ( "CO_NCM" ,      StringType,               nullable = false ) ,
		StructField ( "CO_UNID" ,     StringType,              nullable = true  ) ,
		StructField ( "CO_PAIS" ,     StringType,              nullable = false ) ,
		StructField ( "SG_UF_NCM" ,   StringType ,              nullable = true  ) ,
		StructField ( "CO_VIA" ,      StringType,              nullable = true  ) ,
		StructField ( "CO_URF" ,      StringType,              nullable = true  ) ,
		StructField ( "QT_ESTAT" ,    DecimalType ( 18 , 2 ) ,  nullable = true  ) ,
		StructField ( "KG_LIQUIDO" ,  DecimalType ( 18 , 3 ) ,  nullable = true  ) ,
		StructField ( "VL_FOB" ,      DecimalType ( 18 , 2 ) ,  nullable = true  ) ,
		StructField ( "VL_FRETE" ,    DecimalType ( 18 , 2 ) ,  nullable = true  ) ,
		StructField ( "VL_SEGURO" ,   DecimalType ( 18 , 2 ) ,  nullable = true  ) ,
		StructField ( "TS_REF" ,      TimestampType ,           nullable = false ) ,
		StructField ( "NM_ORIGEM" ,   StringType ,              nullable = false )
  )
)

## Extração
> #### **saprk.read**

In [0]:
val dfs: Seq[DataFrame] = sourcePaths.map { p =>
  println(s"Lendo Delta: $p")
  spark.read.format("delta").load(p)
}

val dfBronzeRaw = dfs . reduce ( _ . unionByName ( _ ) )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_ANO", col("CO_ANO").cast(IntegerType))
  .withColumn("CO_MES", col("CO_MES").cast(IntegerType))
  .withColumn("CO_NCM", trim(upper(col("CO_NCM").cast(StringType))))
  .withColumn("CO_UNID", col("CO_UNID").cast(StringType))
  .withColumn("CO_PAIS", col("CO_PAIS").cast(StringType))
  .withColumn("SG_UF_NCM", trim(upper(col("SG_UF_NCM").cast(StringType))))
  .withColumn("CO_VIA", col("CO_VIA").cast(StringType))
  .withColumn("CO_URF", col("CO_URF").cast(StringType))
  .withColumn("QT_ESTAT", col("QT_ESTAT").cast(DecimalType(18,2)))
  .withColumn("KG_LIQUIDO", col("KG_LIQUIDO").cast(DecimalType(18,3)))
  .withColumn("VL_FOB", col("VL_FOB").cast(DecimalType(18,2)))
  .withColumn("VL_FRETE", col("VL_FRETE").cast(DecimalType(18,2)))
  .withColumn("VL_SEGURO", col("VL_SEGURO").cast(DecimalType(18,2)))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("QT_ESTAT", when(col("QT_ESTAT") < lit(0), lit(0)).otherwise(col("QT_ESTAT")))
  .withColumn("KG_LIQUIDO", when(col("KG_LIQUIDO") < lit(0), lit(0)).otherwise(col("KG_LIQUIDO")))
  .withColumn("VL_FOB", when(col("VL_FOB") < lit(0), lit(0)).otherwise(col("VL_FOB")))
  .withColumn("VL_FRETE", when(col("VL_FRETE") < lit(0), lit(0)).otherwise(col("VL_FRETE")))
  .withColumn("VL_SEGURO", when(col("VL_SEGURO") < lit(0), lit(0)).otherwise(col("VL_SEGURO")))
  .withColumn("SG_UF_NCM", when(length(col("SG_UF_NCM")) === 0, lit(null).cast(StringType)).otherwise(col("SG_UF_NCM")))
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/IMP_*_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
val dfValid = dfWithDefaults . filter ( 
  col("CO_ANO") . isNotNull && 
  col("CO_MES") . isNotNull && 
  col("CO_NCM") . isNotNull && 
  col("CO_PAIS") . isNotNull
) . filter (
  col("CO_MES") . between ( 1 , 12 )
)


In [0]:
val dfDedup = dfValid . dropDuplicates (
  Seq (
    "CO_ANO","CO_MES","CO_NCM","CO_UNID","CO_PAIS","SG_UF_NCM","CO_VIA","CO_URF"
  )
)


In [0]:
val dfSilver = spark.createDataFrame(dfDedup.rdd, silverSchema.fieldNames.foldLeft(dfDedup.schema)((schema, name) => schema))

##Carga
> #### **overwrite**

In [0]:
val deltaTarget = DeltaTable . forName ( "silver_comercio_ext_estatisticas.tb_importacoes" )

deltaTarget.as("t")
  .merge(
    dfSilver.as("s"),
    """
      t.CO_ANO = s.CO_ANO AND
      t.CO_MES = s.CO_MES AND
      t.CO_NCM  = s.CO_NCM AND
      t.CO_UNID = s.CO_UNID AND
      t.CO_PAIS = s.CO_PAIS AND
      t.SG_UF_NCM = s.SG_UF_NCM AND
      t.CO_VIA = s.CO_VIA AND
      t.CO_URF = s.CO_URF
    """
  )
  .whenMatched()
  .updateExpr(Map(
    "t.QT_ESTAT" -> "s.QT_ESTAT"
    "t.KG_LIQUIDO" -> "s.KG_LIQUIDO",
    "t.VL_FOB" -> "s.VL_FOB",
    "t.VL_FRETE" -> "s.VL_FRETE",
    "t.VL_SEGURO" -> "s.VL_SEGURO",
    "t.TS_REF" -> "s.TS_REF",
    "t.NM_ORIGEM" -> "s.NM_ORIGEM"
  ))
  .whenNotMatched()
  .insertAll()
  .execute()
